<a href="https://colab.research.google.com/github/kerryback/Classic_Tests/blob/main/GibbonsRossShanken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# example returns
import pandas_datareader as pdr
startdate = '1980-01-01'
Rets = pdr.DataReader('25_Portfolios_5x5','famafrench',start=startdate)[0]

# Fama-French factors (original three) and RF
FF = pdr.DataReader('F-F_Research_Data_Factors','famafrench',start=startdate)[0]
Facts = FF[['Mkt-RF','SMB','HML']]

# excess returns 
Rets = Rets.apply(lambda x: x-FF['RF'])

# max squared Sharpe ratio from factors
SqShFacts = Facts.mean().T @ np.linalg.inv(Facts.cov()) @ Facts.mean()

# max squared Sharpe ratio from factors + returns
Total = Rets.join(Facts,how='left')
SqShTotal = Total.mean().T @ np.linalg.inv(Total.cov()) @ Total.mean()

# max squared Sharpe ratio of alpha+epsilon
SqShAlphaEps = SqShTotal - SqShFacts

# Gibbons-Ross-Shanken statistic
T = Rets.shape[0]
N = Rets.shape[1]
L = Facts.shape[1]
GRS = (T/N) * ((T-N-L)/(T-L-1)) * SqShAlphaEps / (1+SqShFacts)
print('GRS stat is ' + repr(GRS))

# p value
from scipy.stats import f
pval = 1 - f.cdf(GRS,N,T-N-L)
print('p-value is ' + repr(pval))

# Overview

We illustrate the Gibbons-Ross-Shanken (GRS) test.  We work with factors that are excess returns and test assets that are excess returns (an excess return is a long-minus-short return, where the short could be the risk-free asset).  The test is based on the contribution that the test assets make to the maximum squared Sharpe ratio achievable with the factors.  The maximum squared Sharpe ratio of any set of excess returns is $\mu'\Sigma^{-1}\mu$, where $\mu$ is the vector of means and $\Sigma$ is the covariance matrix.  By regressing the test assets on the factors, we can decompose them as factor risks plus what is left over, which is $\alpha+\varepsilon$ from the regression.  The factor risks do not contribute anything extra to the maximum squared Sharpe ratio of the factors, and the $\alpha+\varepsilon$ is orthogonal to the factors.  Consequently, it can be shown that the maximum squared Sharpe ratio achievable from the combined assets (test assets plus factors) is
$$\mu_C' \Sigma_C^{-1} \mu_C = \mu_F' \Sigma_F^{-1} \mu_F  + \alpha'\Sigma_\varepsilon^{-1}\alpha $$
where $F$ denotes the factors, $\alpha$ is the vector of alphas of the test assets, and $\Sigma_\varepsilon$ is the covariance matrix of the regression residuals.

The GRS statistic is
$$ \frac{T}{N} \cdot \frac{T-N-L}{T-L-1} \cdot \frac{\alpha'\Sigma_\varepsilon^{-1}\alpha}{1+\mu_F' \Sigma_F^{-1} \mu_F} = \frac{T}{N} \cdot \frac{T-N-L}{T-L-1} \cdot \frac{\mu_C' \Sigma_C^{-1} \mu_C - \mu_F' \Sigma_F^{-1} \mu_F}{1+\mu_F' \Sigma_F^{-1} \mu_F}$$
where $T$ is the length of the sample (number of time periods), $N$ is the number of test assets, and $L$ is the number of factors.  Under the null hypothesis that the factor model holds and under a normality assumption, the GRS statistic has an $F$ distribution with $N$ and $T-N-L$ degrees of freedom.

In [2]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import statsmodels.api as sm
from scipy.stats import f

# Define the GRS test


In [3]:
def grs(d,factors) :
  def sqSh(df) :
      return df.mean() @ np.linalg.solve(df.cov(),df.mean())
  C = sqSh(d)
  F = sqSh(d[factors])
  T = d.shape[0]
  L = len(factors)
  N = d.shape[1] - L
  stat = T*(T-N-L)*(C-F) / (N*(T-L-1)*(1+F))
  pval = 1 - f.cdf(stat,N,T-N-L)
  return stat, pval


# Read the file created and saved in the Black_Jensen_Scholes notebook.

In [42]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/crsp_compustat_example2.csv', parse_dates=['date'])
df.date = df.date.dt.to_period('M')
df = df.sort_values(by=['permno','date'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
pd.options.display.max_info_rows = df.shape[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2029402 entries, 0 to 2029401
Data columns (total 16 columns):
 #   Column      Non-Null Count    Dtype    
---  ------      --------------    -----    
 0   Unnamed: 0  2029402 non-null  int64    
 1   permno      2029402 non-null  int64    
 2   permco      2029402 non-null  int64    
 3   date        2029402 non-null  period[M]
 4   ret         2029402 non-null  float64  
 5   me          2029402 non-null  float64  
 6   exchcd      2029402 non-null  int64    
 7   siccd       2029402 non-null  float64  
 8   ticker      2028011 non-null  object   
 9   op          1325538 non-null  float64  
 10  inv         1322038 non-null  float64  
 11  size        2027931 non-null  float64  
 12  bm          1330031 non-null  float64  
 13  Mkt-RF      2029402 non-null  float64  
 14  RF          2029402 non-null  float64  
 15  beta        2029402 non-null  float64  
dtypes: float64(10), int64(4), object(1), period[M](1)
memory usage: 263.2+ M

In [49]:
x = df.groupby('date').apply(lambda d: d.count())
x

,Unnamed: 0,permno,permco,date,ret,me,exchcd,siccd,ticker,op,inv,size,bm,Mkt-RF,RF,beta
date,,,,,,,,,,,,,,,,
1967-02,973,973,973,973,973,973,973,973,973,722,718,973,728,973,973,973
1967-03,974,974,974,974,974,974,974,974,974,723,720,974,730,974,974,974
1967-04,973,973,973,973,973,973,973,973,973,724,721,973,731,973,973,973
1967-05,969,969,969,969,969,969,969,969,969,725,722,969,732,969,969,969
1967-06,969,969,969,969,969,969,969,969,969,727,724,969,734,969,969,969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08,2692,2692,2692,2692,2692,2692,2692,2692,2692,2486,2473,2692,2487,2692,2692,2692
2021-09,2693,2693,2693,2693,2693,2693,2693,2693,2693,2486,2473,2693,2487,2693,2693,2693
2021-10,2693,2693,2693,2693,2693,2693,2693,2693,2693,2485,2472,2693,2486,2693,2693,2693


In [53]:
x.loc['1973-07':'1975-07']

,Unnamed: 0,permno,permco,date,ret,me,exchcd,siccd,ticker,op,inv,size,bm,Mkt-RF,RF,beta
date,,,,,,,,,,,,,,,,
1973-07,1716,1716,1716,1716,1716,1716,1716,1716,1716,2,1,1716,2,1716,1716,1716
1973-08,1725,1725,1725,1725,1725,1725,1725,1725,1725,2,1,1725,2,1725,1725,1725
1973-09,1735,1735,1735,1735,1735,1735,1735,1735,1735,1,1,1735,1,1735,1735,1735
1973-10,1746,1746,1746,1746,1746,1746,1746,1746,1746,0,0,1745,0,1746,1746,1746
1973-11,1750,1750,1750,1750,1750,1750,1750,1750,1750,1,1,1749,1,1750,1750,1750
1973-12,1764,1764,1764,1764,1764,1764,1764,1764,1764,1,1,1763,1,1764,1764,1764
1974-01,1768,1768,1768,1768,1768,1768,1768,1768,1768,1,1,1767,1,1768,1768,1768
1974-02,1777,1777,1777,1777,1777,1777,1777,1777,1777,1,1,1776,1,1777,1777,1777
1974-03,1793,1793,1793,1793,1793,1793,1793,1793,1793,1,1,1792,1,1793,1793,1793


# Read the original Fama-French factors

In [5]:
ff = pdr('F-F_Research_Data_Factors','famafrench',start='1962-01-01')[0] / 100

# Example 1: Test the CAPM with 10 beta-sorted portfolios

In [6]:
# compute decile returns
df['decile'] = df.groupby('date').beta.apply(lambda x: pd.qcut(x,10,labels=range(1,11)))
rets = df.groupby(['date','decile']).apply(lambda d: (d.me*d.ret).sum() / d.me.sum())
rets = rets.unstack()
df = df.drop(columns=['decile'])

# add market excess return
rets['Mkt-RF'] = ff['Mkt-RF']
rets = rets.dropna()

# run the GRS test
grs(rets,['Mkt-RF'])


(14.62156487584466, 1.1102230246251565e-16)

# Example 2: Test the original FF model with 10 operating profit sorted portfolios

In [24]:
x = df[df.date=='1975-03']

In [27]:
df2 = df.dropna(subset=['op'])

In [32]:
def cut(d,x,n) :
  try :
    return pd.qcut(d[x],n,labels=range(1,n+1))
  except :
    print(d.date.iloc[0])
    return pd.Series(np.nan,index=d.index)

In [41]:
df2 = df.dropna(subset=['beta','me','ret'],how='any')
x = df2[df2.date=='2020-05']
x.shape

(2668, 16)

In [39]:
x.shape

(1, 16)

In [ ]:
x.op.unique()

In [33]:
df2.groupby('date').apply(lambda d: cut(d,'op',10))

1973-09
1973-11
1973-12
1974-01
1974-02
1974-03
1974-05
1974-06
1974-07
1974-08
1974-09
1974-10
1974-11
1974-12
1975-03
1975-06
1979-09
1979-10
1985-06
1985-07
1986-01
1986-06
1991-07
1991-08
1991-09
1992-02
1992-03
1992-05
2002-04
2002-05
2002-10
2012-08
2012-11
2012-12
2013-01
2013-03
2013-04
2013-05
2013-08
2013-09
2013-10
2014-01
2018-07
2018-08
2018-12
2019-01
2019-11
2019-12
2020-01
2020-02
2020-03
2020-04
2020-05


date            
1967-02  645        7.0
         1251       1.0
         5126       6.0
         7894       4.0
         10402      8.0
                   ... 
2021-12  2028523    7.0
         2028735    3.0
         2029159    5.0
         2029323    2.0
         2029401    6.0
Length: 1325538, dtype: float64

In [31]:
# compute decile returns 

df2 = df.dropna(subset=['inv','me','ret'],how='any')

df2['decile'] = df2.groupby('date').op.apply(lambda x: pd.qcut(x,10,labels=range(1,11)))
rets = df2.groupby(['date','decile']).apply(lambda d: (d.me*d.ret).sum() / d.me.sum())
rets = rets.unstack()

# add Fama-French factors
rets = rets.merge(ff[['Mkt-RF','SMB','HML']],left_index=True,right_index=True,how='inner')

# run the GRS test
grs(rets,['Mkt-RF','SMB','HML'])

ValueError: ignored

In [14]:
df.groupby('date').inv.apply(lambda x: len(x.unique()))

date
1967-02     717
1967-03     719
1967-04     720
1967-05     721
1967-06     723
           ... 
2021-08    2474
2021-09    2474
2021-10    2473
2021-11    2472
2021-12    2468
Freq: M, Name: inv, Length: 659, dtype: int64

In [22]:
df[df.permno==34198].iloc[59:100]

,Unnamed: 0,permno,permco,date,ret,me,exchcd,siccd,ticker,op,inv,size,bm,Mkt-RF,RF,beta
645119,1022373,34198,23982,1972-07,-0.055556,6345.0000,2,5621.0,MLW,0.251987,0.242548,6345.000,0.000886,-0.0080,0.0031,1.585590
645120,1022374,34198,23982,1972-08,-0.078431,5992.5000,2,5621.0,MLW,0.251987,0.242548,6345.000,0.000886,0.0326,0.0029,1.517155
645121,1022376,34198,23982,1974-07,0.469388,5555.3750,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.0805,0.0070,1.531341
645122,1022377,34198,23982,1974-08,-0.030556,8163.0000,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.0935,0.0060,1.539023
645123,1022378,34198,23982,1974-09,-0.166667,7822.8750,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.1177,0.0081,1.112570
645124,1022379,34198,23982,1974-10,0.243478,6519.0625,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,0.1610,0.0051,1.059478
645125,1022380,34198,23982,1974-11,0.004196,8106.3125,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.0451,0.0054,1.079245
645126,1022381,34198,23982,1974-12,-0.140845,7339.6250,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.0345,0.0070,1.143752
645127,1022382,34198,23982,1975-01,0.377049,6305.8750,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,0.1366,0.0058,1.119164
645128,1022383,34198,23982,1975-02,0.226190,8683.5000,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,0.0556,0.0043,1.145330


In [16]:
df[df.op.round(8)==0.25198719]

,Unnamed: 0,permno,permco,date,ret,me,exchcd,siccd,ticker,op,inv,size,bm,Mkt-RF,RF,beta
645119,1022373,34198,23982,1972-07,-0.055556,6345.0000,2,5621.0,MLW,0.251987,0.242548,6345.000,0.000886,-0.0080,0.0031,1.585590
645120,1022374,34198,23982,1972-08,-0.078431,5992.5000,2,5621.0,MLW,0.251987,0.242548,6345.000,0.000886,0.0326,0.0029,1.517155
645121,1022376,34198,23982,1974-07,0.469388,5555.3750,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.0805,0.0070,1.531341
645122,1022377,34198,23982,1974-08,-0.030556,8163.0000,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.0935,0.0060,1.539023
645123,1022378,34198,23982,1974-09,-0.166667,7822.8750,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.1177,0.0081,1.112570
645124,1022379,34198,23982,1974-10,0.243478,6519.0625,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,0.1610,0.0051,1.059478
645125,1022380,34198,23982,1974-11,0.004196,8106.3125,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.0451,0.0054,1.079245
645126,1022381,34198,23982,1974-12,-0.140845,7339.6250,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,-0.0345,0.0070,1.143752
645127,1022382,34198,23982,1975-01,0.377049,6305.8750,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,0.1366,0.0058,1.119164
645128,1022383,34198,23982,1975-02,0.226190,8683.5000,2,0.0,MLW,0.251987,0.242548,5555.375,0.000886,0.0556,0.0043,1.145330
